In [1]:
import pandas as ps
import numpy as np 
import os
import re

In [2]:
os.listdir("data")

['Flore du Cameroun tome 10 1970 Ombellales',
 'Flore du Cameroun tome 10 1970 Ombellales.pdf',
 'Flore du Cameroun tome 12 1972 Loganiaceae',
 'Flore du Cameroun tome 12 1972 Loganiaceae.pdf',
 'Flore du Cameroun tome 16 1973 Sapindaceae',
 'Flore du Cameroun tome 16 1973 Sapindaceae.pdf',
 'Flore du Cameroun tome 17 1974 Amaranthaceae',
 'Flore du Cameroun tome 17 1974 Amaranthaceae.pdf',
 'Flore du Cameroun tome 2 1964 Sapotacees',
 'Flore du Cameroun tome 2 1964 Sapotacees.pdf',
 'Flore du Cameroun tome 6 1967 Cucurbitaceae',
 'Flore du Cameroun tome 6 1967 Cucurbitaceae.pdf',
 'Flore du Cameroun tome 9 1970 Caesalpinioidae',
 'Flore du Cameroun tome 9 1970 Caesalpinioidae.pdf']

In [3]:
folder = []
for item in os.listdir('data'):
    if(len(item.split('.')) == 1):
        folder.append(item)

In [4]:
folder

['Flore du Cameroun tome 10 1970 Ombellales',
 'Flore du Cameroun tome 12 1972 Loganiaceae',
 'Flore du Cameroun tome 16 1973 Sapindaceae',
 'Flore du Cameroun tome 17 1974 Amaranthaceae',
 'Flore du Cameroun tome 2 1964 Sapotacees',
 'Flore du Cameroun tome 6 1967 Cucurbitaceae',
 'Flore du Cameroun tome 9 1970 Caesalpinioidae']

In [5]:
def f_remove_accents(old):
    """
    Removes common accent characters, lower form.
    Uses: regex.
    """
    new = old.lower()
    new = re.sub(r'[àáâãäå]', 'a', new)
    new = re.sub(r'[èéêë]', 'e', new)
    new = re.sub(r'[ìíîï]', 'i', new)
    new = re.sub(r'[òóôõö]', 'o', new)
    new = re.sub(r'[ùúûü]', 'u', new)
    return new
f_remove_accents("CLÉ DES ESPÈCES CAMEROUNAISES ET GABONAISES")


'cle des especes camerounaises et gabonaises'

# 1 - Extract species names

In [6]:
def especes_pages(path):
    pages = []
    for i in range(len(os.listdir(path))):
        with open(path+'/'+ str(i+1) +'.txt', 'r') as fp:
            lines = fp.readlines()
            #print("=================")
            for j in range(len(lines)):
                l = lines[j].strip()
                l = l.replace('  ',' ')
                l = f_remove_accents(l)
                x = re.search("cle.?\sdes\sespece", l)
                if(x):
                    pages.append(i+1)
                    pages.append(i+2)
                    pages.append(i+3)
    return pages

In [7]:
def get_especes(pages):
    especes = []
    for p in pages:
        with open(path+'/'+ str(p) +'.txt', 'r') as fp:
            lines = fp.readlines()
            for j in range(len(lines)):
                l = lines[j].strip()
                l = l.replace('  ',' ')
                l = f_remove_accents(l)
                x = re.findall("\s\w\.\s\w+$", l)
                x = re.findall("[A-Za-z0-9]\.\s\w\.\s\w+\.?$", l)
                if(x):
                    especes.append(x)
    especes = np.array(especes).reshape(-1)
    esp = [esp.strip().split(' ')[2][:-1] for esp in especes]
    initials = [esp.strip().split(' ')[1][:-1].upper() for esp in especes]
    return esp, initials

def preprocess(line):
    l = line.strip()
    l = l.replace('  ',' ')
    #l = f_remove_accents(l)
    return l

## get species full name

In [8]:
def get_especes_names(especes, initials, path):
    esp_names = []
    for i in range(len(os.listdir(path))):
        with open(path+'/'+ str(i+1) +'.txt', 'r') as fp:
            lines = fp.readlines()
            for j in range(len(lines)):
                l = preprocess(lines[j])
                for i, esp in enumerate(especes):
                    contains = l.find(esp.capitalize()) != -1 or l.find(esp) != -1 
                    if contains:
                        exp = "^[1-9]\.\s{}[a-z]+.*".format(initials[i])
                        x = re.findall(exp, l)
                        if(x):
                            full_name = x[0].strip()[3:]
                            full_name = full_name.split(' ')[0]+' '+full_name.split(' ')[1]
                            esp_names.append(full_name)
    return esp_names

## Execute

In [46]:
paths = ['data/'+ fold for fold in folder]
names = []
for path in paths:
    print(path)
    print("======================")
    pages = especes_pages(path)
    especes, initials = get_especes(pages)
    print("Species number : "+ str(len(especes)))
    all_species = get_especes_names(especes,initials, path)
    names += all_species
    print("Species title number : "+ str(len(all_species)))

data/Flore du Cameroun tome 10 1970 Ombellales
Species number : 17
Species title number : 16
data/Flore du Cameroun tome 12 1972 Loganiaceae
Species number : 63
Species title number : 43
data/Flore du Cameroun tome 16 1973 Sapindaceae
Species number : 62
Species title number : 41
data/Flore du Cameroun tome 17 1974 Amaranthaceae
Species number : 21
Species title number : 17
data/Flore du Cameroun tome 2 1964 Sapotacees
Species number : 33
Species title number : 33
data/Flore du Cameroun tome 6 1967 Cucurbitaceae
Species number : 37
Species title number : 30
data/Flore du Cameroun tome 9 1970 Caesalpinioidae
Species number : 92
Species title number : 69


In [47]:
len(set(names))

213

In [48]:
all_species

['Griffonia physocarpa',
 'Griffonia speciosa',
 'Piliostigma Thonningii',
 'Piliostigma reticulatum',
 'Amphimas ferrugineus',
 'Amphimas pterocarpoides',
 'Dialium Dinklagei',
 'Dialium Zenkeri',
 'Cassia Mannii',
 'Cassia mimosoides',
 'Cassia Kirkii',
 'Cassia alata',
 'Cassia italica',
 'Cassia Sieberana',
 'Cassia obtusifolia',
 'Gilletiodendron Mildbraedii',
 'Cynometra Mannii',
 'Plagiosiphon emarginatus',
 'Plagiosiphon longitubus',
 'Plagiosiphon multijugus',
 'Plagiosiphon discifer',
 'Hymenostegia brachyura',
 'Hymenostegia Breteleri',
 'Hymenostegia Felicis',
 'Afzelia africana',
 'Oxystigma Buchholzii',
 'Copaifera Mildbraedii',
 'Copaifera religiosa',
 'Detarium microcarpum',
 'Detarium macrocarpum',
 'Tessmannia anomala',
 'Tessmannia africana',
 'Daniellia Oliveri',
 'Daniellia oblonga',
 'Isomacrolobium leptorrhachis',
 'Isomacrolobium isopetalum',
 'Anthonotha ferruginea',
 'Anthonotha lamprophylla',
 'Anthonotha fragrans',
 'Gilbertiodendron ogoouense',
 'Gilbertiod

# =================================
# 2 - Find organes and descriptors
# =================================

In [9]:
from bs4 import BeautifulSoup as bs
import requests

url="https://fr.wikipedia.org/wiki/Glossaire_de_botanique"
response = requests.get(url)

html = response.content

soup = bs(html, 'lxml')


In [10]:
def get_glossaire(soup):
    glossaire = []
    div = soup.find("div", {"class" : "mw-parser-output"})
    uls = div.find_all("ul")
    for i, ul in enumerate(uls):
        lis = ul.find_all("li")
        for li in lis:
            if li.find("b") != None:
                noun = li.get_text().split(':')[0][:-1].strip()
                if re.search("adjectif",li.get_text()) or re.search("se dit\s",li.get_text()) or re.search("é$",noun) or re.search("qualifie\s",li.get_text()):
                    glossaire.append((noun.split(' ')[0].lower(), "adj"))
                    #print(noun.strip().split(' ')[0].lower() + ': adj')
                else:
                    glossaire.append((noun.split(' ')[0].lower(), "noun"))
                    #print(noun.strip().split(' ')[0].lower() + ': adj')
    return glossaire

In [11]:
glossaire = get_glossaire(soup)

In [132]:
organes = [
    "racine",
    "tige",
    "bourgeon",
    "rhizome",
    "tubercule",
    "bulbe",
    "collet",
    "limbe",
    "lobe",
    "pétiole",
    "sessile",
    "foliole",
    "calice",
    "sépale",
    "calicule",
    "corolle",
    "anthère"
    "tépale",
    "stipule",
    "foliole",
    "carpelle",
    "grappe ",
    "vrille",
    "feuille",
    "fleur"
]

In [13]:
adjs = [gls[0].lower() for gls in glossaire if gls[1] == "adj"]
nouns = [gls[0].lower() for gls in glossaire if gls[1] == "noun"]

In [14]:
adjs = adjs[:339] + adjs[341:]

## TEST ===============

In [15]:
from nltk import pos_tag, word_tokenize
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer(language='french')

In [16]:
text = """
Plante  herbacée  grimpante  dioïque  à tiges  grêles,  sillonnées, 
glabres. 
Feuilles  pétiolées;  pétiole  grêle,  légèrement  pubescent  ou 
glabre,  portant  à son  sommet  deux  glandes  coniques,  longues  de 
7-20 mm;  limbe  membraneux  ovale-oblong  ou largement  ovale, 
5-11  X 4"7  cm,  glabre  sur les deux  faces,  tronqué  à la base  ou très 
légèrement  émarginé,  parfois  3-5-lobé,  lobes  aigus  ou apiculés, 
triangulaires,  bords  faiblement  sinueux-dentés.  Des  trois  nervures 
basilaires,  les deux  latérales  se bifurquent  très  tôt;  nervures  net-
tement  saillantes  à la face  inférieure,  marquées  à la face  supérieure. 
Vrilles  grêles,  simples,  glabres. 
Inflorescences  ¿ en racèmes  pauciflores,  6-10  fleurs  environ, 
fleurs  blanches,  axes  des racèmes  atteignant  8-12  cm de longueur, 
glabres;  pédicelles  glabres,  longs  de 5-20  mm,  bractée  minuscule. 
Coupe  florale  très  allongée,  globuleuse  à la base,  puis  rétrécie  et 
enfin  cylindrique,  atteignant  5o mm  de longueur,  glabre  à la face 
externe,  villeuse  sur  la face  interne,  10 mm  de diamètre  à la 
gorge.  Sépales  linéaires  aigus,  longs  de 4-5  mm  glabres,  portant 
quelques  glandes  sur leurs  bords.  Pétales  obovales,  finement  pubes-
cents  sur les deux  faces,  à bords  plus  ou moins  enroulés,  longs  de 
2-3 cm.  Anthères  cohérentes  au milieu  de la coupe  florale,  loges 
condupliquées,  longues  de I,5-2  cm,  filets  très  courts. 
"""

In [17]:
from nltk import sent_tokenize

In [18]:
sentences = sent_tokenize(text)

In [19]:
###===============================
# utils functions 
###===============================

def prepare(sentence):
    sent = sentence.replace("\n", "").lower()
    sent = preprocess(sent)
    sent = " ".join(word_tokenize(sent))
    return sent

def did_overlap(pos, begin, end):
    for i, p in enumerate(pos):
        if p[0] == begin or p[1] == end:
            return True
    return False

def fix_overlap(temp):
    for i in range(len(temp)-1):
        for j in temp[i+1:]:
            if temp[i][0] == j[0] or temp[i][1] == j[1]:
                len_i = temp[i][1] - temp[i][0]
                len_j = j[1] - j[0]
                if(len_i > len_j):
                    temp.remove(j)
                else:
                    temp.remove(temp[i])
    return temp

def get_measure_begining(sent, last_index):
    last = last_index
    while True:
        virg_ind = sent[:last].rfind(",")
        if sent[virg_ind - 1].isdigit():
            last = virg_ind
        else:
            break
    begin = virg_ind + re.search("\d+",sent[virg_ind:]).span()[0]
    return begin

In [138]:
#Extract organs
def organe_(sent):
    pos = []
    tokens = word_tokenize(sent)
    for org in organes:
        stem = org
        start = [m.start() for m in re.finditer(stem, sent)]
        if len(start) != 0:
            for st in start:
                begin = st
                end = st + sent[begin:].find(' ')
                if did_overlap(pos, begin, end):
                    continue
                else:
                    pos.append((begin, end, 'ORGAN' ))
    return pos
#Extract descriptor
def descripteur(sent):
    pos = []
    tokens = word_tokenize(sent)
    for adj in adjs:
        stem = adj
        start = [m.start() for m in re.finditer(stem, sent)]
        if len(start) != 0:
            for st in start:
                begin = st
                end = st + sent[begin:].find(' ')
                if did_overlap(pos, begin, end):
                    continue
                else:
                    pos.append((begin, end, 'DESC' ))
    # try to find descriptors using POS (adjectivs ends with é,ée,és,ées)
    start = [m.start() for m in re.finditer("\w+é\s|\w+ée\s|\w+és\s|\w+ées\s", sent)]
    if len(start) != 0:
        for st in start:
            begin = st
            end = st + sent[begin:].find(' ')
            if did_overlap(pos, begin, end):
                continue
            else:
                pos.append((begin, end, 'DESC' ))
    pos = list(set(pos))
    return pos

#Extract measure
def measure(sent):
    pos = []
    tokens = word_tokenize(sent)
    if(re.search(" m | cm | mm ", sent)):
        measures = re.findall(" m | cm | mm ", sent)
        start = [m.start() for m in re.finditer(" m | cm | mm ", sent)]
        for i, st in enumerate(start):
            begin = get_measure_begining(sent, st)
            end = st + len(measures[i]) -1    
            if did_overlap(pos, begin, end):
                continue
            else:
                pos.append((begin, end, 'MEASURE' ))
    return pos

# Extract colors

In [21]:
colors = [
    "rouge",
    "rose",
    "bleu",
    "pourpre",
    "violet",
    "jaune",
    "orange",
    "crème",
    "blanc",
    "blanche",
    "vert",
    "brun"
]

In [22]:
sentences = sent_tokenize(text)

for sent in sentences:
    sent = prepare(sent)

In [23]:
#Extract color
# search in color list
def color_1(sent):
    pos = []
    tokens = word_tokenize(sent)
    for c in colors:
        start = [m.start() for m in re.finditer(c , sent)]
        if len(start) != 0:
            for st in start:
                begin = st
                # solve the problem of finding DESC & COLOR in the same Entity
                if sent[begin:].find('-') == -1:
                    end = begin + sent[begin:].find(' ')
                else:
                    
                    end = begin + min(sent[begin:].find(' '),sent[begin:].find('-'))
                # for example verticalisé is classified as a color so we check its length 
                if did_overlap(pos, begin, end) or ((end - begin) > len(c)+2) :
                    continue
                else:
                    pos.append((begin, end, 'COLOR' ))
    return pos
# word ends with âtre
def color_2(sent):
    pos = []
    tokens = word_tokenize(sent)
    for c in colors:
        stem = c
        start = [m.start() for m in re.finditer("\s\w+âtre|\s\w+âtres" , sent)]
        if len(start) != 0:
            for st in start:
                begin = st+1
                end = begin + sent[begin:].find(' ')
                if did_overlap(pos, begin, end):
                    continue
                else:
                    pos.append((begin, end, 'COLOR' ))
    return pos
# combine both functions
def color(sent):
    pos = color_1(sent) + color_2(sent)
    pos = fix_overlap(pos)
    return pos

In [25]:
sentences

['\nPlante  herbacée  grimpante  dioïque  à tiges  grêles,  sillonnées, \nglabres.',
 'Feuilles  pétiolées;  pétiole  grêle,  légèrement  pubescent  ou \nglabre,  portant  à son  sommet  deux  glandes  coniques,  longues  de \n7-20 mm;  limbe  membraneux  ovale-oblong  ou largement  ovale, \n5-11  X 4"7  cm,  glabre  sur les deux  faces,  tronqué  à la base  ou très \nlégèrement  émarginé,  parfois  3-5-lobé,  lobes  aigus  ou apiculés, \ntriangulaires,  bords  faiblement  sinueux-dentés.',
 'Des  trois  nervures \nbasilaires,  les deux  latérales  se bifurquent  très  tôt;  nervures  net-\ntement  saillantes  à la face  inférieure,  marquées  à la face  supérieure.',
 'Vrilles  grêles,  simples,  glabres.',
 'Inflorescences  ¿ en racèmes  pauciflores,  6-10  fleurs  environ, \nfleurs  blanches,  axes  des racèmes  atteignant  8-12  cm de longueur, \nglabres;  pédicelles  glabres,  longs  de 5-20  mm,  bractée  minuscule.',
 'Coupe  florale  très  allongée,  globuleuse  à la base,  pui

In [133]:
sent = sentences[4]
sent

'Inflorescences  ¿ en racèmes  pauciflores,  6-10  fleurs  environ, \nfleurs  blanches,  axes  des racèmes  atteignant  8-12  cm de longueur, \nglabres;  pédicelles  glabres,  longs  de 5-20  mm,  bractée  minuscule.'

In [134]:
sent = prepare(sent)

In [135]:
sent

'inflorescences ¿ en racèmes pauciflores , 6-10 fleurs environ , fleurs blanches , axes des racèmes atteignant 8-12 cm de longueur , glabres ; pédicelles glabres , longs de 5-20 mm , bractée minuscule .'

In [136]:
res = descripteur(sent) + organe_(sent) + measure(sent) + list(set(color(sent)))
res

[(132, 139, 'DESC'),
 (153, 160, 'DESC'),
 (182, 189, 'DESC'),
 (47, 53, 'ORGAN'),
 (64, 70, 'ORGAN'),
 (110, 117, 'MEASURE'),
 (172, 179, 'MEASURE'),
 (71, 79, 'COLOR')]

In [137]:
for r in res:
    print(sent[r[0] : r[1]] , r[2])

glabres DESC
glabres DESC
bractée DESC
fleurs ORGAN
fleurs ORGAN
8-12 cm MEASURE
5-20 mm MEASURE
blanches COLOR


In [126]:
re.findall("\w+é\s|\w+ée\s|\w+és\s|\w+ées\s", sent)

[]

In [107]:
# try to find descriptors using POS (adjectivs ends with é,ée,és,ées)

start = [m.start() for m in re.finditer("\w+é\s|\w+ée\s|\w+és\s|\w+ées\s", sent)]
start

[9, 234, 271, 294, 316, 368]

In [120]:
sent[130:138]

'marquées'

##  Create training dataset

In [407]:
import random

random.shuffle(folder)

In [408]:
folder

['Flore du Cameroun tome 16 1973 Sapindaceae',
 'Flore du Cameroun tome 12 1972 Loganiaceae',
 'Flore du Cameroun tome 6 1967 Cucurbitaceae',
 'Flore du Cameroun tome 9 1970 Caesalpinioidae',
 'Flore du Cameroun tome 10 1970 Ombellales',
 'Flore du Cameroun tome 2 1964 Sapotacees',
 'Flore du Cameroun tome 17 1974 Amaranthaceae']

In [409]:
folder_train, folder_test = folder[:5], folder[5:]

In [411]:
folder_test

['Flore du Cameroun tome 2 1964 Sapotacees',
 'Flore du Cameroun tome 17 1974 Amaranthaceae']

In [412]:
%%time

DATA = []
for f in folder_train:
    path = 'data/' + f
    for i in range(len(os.listdir(path))):
        with open(path+'/'+ str(i+1) +'.txt', 'r') as fp:
            lines = fp.read()
            sentences = sent_tokenize(lines)

            for sent in sentences:
                sent = prepare(sent)
                sent = " ".join(word_tokenize(sent))
                res = descripteur(sent) + organe_(sent) + measure(sent) + list(set(color(sent)))
                if len(res) != 0:
                    res = fix_overlap(res)
                    res = {'entities' : res}
                    result = (sent , res)
                    DATA.append(result)


Wall time: 1min 28s


In [425]:
import spacy
from spacy.tokens import DocBin
from pathlib import Path
import warnings

def convert(lang: str, TRAIN_DATA, output_path: Path):
    nlp = spacy.blank(lang)
    db = DocBin()
    for text, annot in TRAIN_DATA:
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label)
            if span is None:
                msg = f"Skipping entity [{start}, {end}, {label}] in the following text because the character span '{doc.text[start:end]}' does not align with token boundaries:\n\n{repr(text)}\n"
                warnings.warn(msg)
            else:
                ents.append(span)
        doc.ents = ents
        db.add(doc)
    db.to_disk(output_path)

In [426]:
convert("fr", DATA, "ner_data/train.spacy")

C:\Users\NEW-USER\AppData\Local\Temp/ipykernel_23556/676142066.py:16: UserWarning: Skipping entity [201, 212, DESC] in the following text because the character span 'trimoinedbd' does not align with token boundaries:

"7/ pour toute autre question relative à la réutilisation des documents numérisés par le mnhn , l'utilisateur est invité à s'informer auprès de la direction des bibliothèques et de la documentation : patrimoinedbd @ mnhn.fr"

  warnings.warn(msg)
C:\Users\NEW-USER\AppData\Local\Temp/ipykernel_23556/676142066.py:16: UserWarning: Skipping entity [38, 42, DESC] in the following text because the character span 'trie' does not align with token boundaries:

"fleurs zygomorphes , avec plan de symétrie du disque et de l'androcée oblique , ou actinomorphes ."

  warnings.warn(msg)
C:\Users\NEW-USER\AppData\Local\Temp/ipykernel_23556/676142066.py:16: UserWarning: Skipping entity [128, 136, DESC] in the following text because the character span 'staminal' does not align with token b

C:\Users\NEW-USER\AppData\Local\Temp/ipykernel_23556/676142066.py:16: UserWarning: Skipping entity [23, 28, DESC] in the following text because the character span 'ovale' does not align with token boundaries:

'fruit subsphérique à obovale , de 5-7 mm .'

  warnings.warn(msg)
C:\Users\NEW-USER\AppData\Local\Temp/ipykernel_23556/676142066.py:16: UserWarning: Skipping entity [9, 12, DESC] in the following text because the character span 'nue' does not align with token boundaries:

"forme connue du sénégal , de guinée , de côte d'ivoire , du nigeria et du ghana ."

  warnings.warn(msg)
C:\Users\NEW-USER\AppData\Local\Temp/ipykernel_23556/676142066.py:16: UserWarning: Skipping entity [10, 17, DESC] in the following text because the character span 'tribuée' does not align with token boundaries:

"espèce distribuée du zaïre et de l'uganda à la zambie dans les gale-ries forestières et les savanes jusqu ' à 2000 m d'altitude ."

  warnings.warn(msg)
C:\Users\NEW-USER\AppData\Local\Temp/ipykern

C:\Users\NEW-USER\AppData\Local\Temp/ipykernel_23556/676142066.py:16: UserWarning: Skipping entity [191, 199, DESC] in the following text because the character span 'opposées' does not align with token boundaries:

'rachis foliaire long de 75 cm dont 31 cm au dessous de la première paire de folioles , cylin-drique de 5 mm de diamètre , finement strié en long à sec ; 5-7 paires de folioles opposées ou subopposées , à pétiolule de 5-12 mm ; limbe glabre , discolore à sec , olivacé dessus , roussâtre dessous , oblong obové , à base aiguë inégale et sommet abruptement acuminé , de 15-40 x 7-14 cm ; nervure médiane glabre dans une légère dépression à la face supérieure ; 14-18 paires de nervures secondaires en relief , ainsi que le fin réticule , sur les deux faces à sec .'

  warnings.warn(msg)
C:\Users\NEW-USER\AppData\Local\Temp/ipykernel_23556/676142066.py:16: UserWarning: Skipping entity [265, 274, DESC] in the following text because the character span 'concolore' does not align with t

C:\Users\NEW-USER\AppData\Local\Temp/ipykernel_23556/676142066.py:16: UserWarning: Skipping entity [12, 19, DESC] in the following text because the character span 'pennées' does not align with token boundaries:

'feuilles bi-pennées , pétiole ( 10-20 cm ) et rachis ( 40-45 cm ) striés ; pinnules opposés ( 9-12 paires ) de 12-'

  warnings.warn(msg)
C:\Users\NEW-USER\AppData\Local\Temp/ipykernel_23556/676142066.py:16: UserWarning: Skipping entity [148, 156, DESC] in the following text because the character span 'tri-ques' does not align with token boundaries:

'22 cm , aplaties dessus , à base épaissie ; tous ces axes couverts de poils étoilés ferrugineux ; 15-20 paires de folioles de 17-20 x 7-8 mm , asymétri-ques , légèrement falciformes , glabres dessus , velues sous la nervure médiane , env .'

  warnings.warn(msg)
C:\Users\NEW-USER\AppData\Local\Temp/ipykernel_23556/676142066.py:16: UserWarning: Skipping entity [61, 66, COLOR] in the following text because the character span 'verts

C:\Users\NEW-USER\AppData\Local\Temp/ipykernel_23556/676142066.py:16: UserWarning: Skipping entity [55, 61, COLOR] in the following text because the character span 'vertes' does not align with token boundaries:

"arbres élevés , 30-40 m , dioïques ; jeunes pousses couvertes de résine ; rameaux glabres ; écorce faisant mousser l'eau ."

  warnings.warn(msg)
C:\Users\NEW-USER\AppData\Local\Temp/ipykernel_23556/676142066.py:16: UserWarning: Skipping entity [92, 99, DESC] in the following text because the character span 'triques' does not align with token boundaries:

"feuilles paripennées , à 5-9 ( -12 ) paires de folioles , le plus souvent alternes , dissymétriques , ± falci-formes , acuminées , entières , membraneuses , glabres , à cellules résinifères dont la sécrétion rend les feuilles brillantes , d'où le nom générique ( du grec ganos = brillant , et phyllum = feuille ) ."

  warnings.warn(msg)
C:\Users\NEW-USER\AppData\Local\Temp/ipykernel_23556/676142066.py:16: UserWarning: Skippin

...

C:\Users\NEW-USER\AppData\Local\Temp/ipykernel_23556/676142066.py:16: UserWarning: Skipping entity [14, 19, DESC] in the following text because the character span 'nueux' does not align with token boundaries:

'fût parfois sinueux .'

  warnings.warn(msg)
C:\Users\NEW-USER\AppData\Local\Temp/ipykernel_23556/676142066.py:16: UserWarning: Skipping entity [150, 156, DESC] in the following text because the character span 'trices' does not align with token boundaries:

"gros fruit ligneux , droit , à valves lisses , de io-3o x 5,5-8 cmv il contient de 5 à 10 graines , chacune placée dans une loge aux * cloisons séparatrices ' bien- marquées ."

  warnings.warn(msg)
C:\Users\NEW-USER\AppData\Local\Temp/ipykernel_23556/676142066.py:16: UserWarning: Skipping entity [85, 92, DESC] in the following text because the character span 'allèles' does not align with token boundaries:

'nervures secondaires réunies en arcs près de la marge et nervilles intermédiaires parallèles , nombreuses , formant av

C:\Users\NEW-USER\AppData\Local\Temp/ipykernel_23556/676142066.py:16: UserWarning: Skipping entity [252, 258, COLOR] in the following text because the character span 'brunii' does not align with token boundaries:

"une dizaine d'espèces d'afrique occidentale et équatoriale : i. elongatum en sierra leone ; i. vignei de sierra leone au ghana ; i. nigericum et i. obanense au nigeria ; i. leptorrhachis au cameroun ; i. graciliflorum au rio muni ; i. sargosii et i. lebrunii au congo ."

  warnings.warn(msg)
C:\Users\NEW-USER\AppData\Local\Temp/ipykernel_23556/676142066.py:16: UserWarning: Skipping entity [10, 13, DESC] in the following text because the character span 'nue' does not align with token boundaries:

'espèce connue du cameroun méridional .'

  warnings.warn(msg)
C:\Users\NEW-USER\AppData\Local\Temp/ipykernel_23556/676142066.py:16: UserWarning: Skipping entity [31, 35, DESC] in the following text because the character span 'lobé' does not align with token boundaries:

'calice à 4 

C:\Users\NEW-USER\AppData\Local\Temp/ipykernel_23556/676142066.py:16: UserWarning: Skipping entity [96, 103, DESC] in the following text because the character span 'tribuée' does not align with token boundaries:

"c'est pourquoi il nous a paru nécessaire de séparer génériquement l'espèce de léonard d'abord attribuée à anthonotha puis à isomacrolobium ."

  warnings.warn(msg)
C:\Users\NEW-USER\AppData\Local\Temp/ipykernel_23556/676142066.py:16: UserWarning: Skipping entity [10, 17, ORGAN] in the following text because the character span 'sessile' does not align with token boundaries:

'ovaire subsessile .'

  warnings.warn(msg)
C:\Users\NEW-USER\AppData\Local\Temp/ipykernel_23556/676142066.py:16: UserWarning: Skipping entity [30, 35, DESC] in the following text because the character span 'trice' does not align with token boundaries:

"caduque et qui laisse une cicatrice intrapétiolaire qui s'étend longuement et très visiblement de part et d'autre sur le rameau ."

  warnings.warn(msg)
C

C:\Users\NEW-USER\AppData\Local\Temp/ipykernel_23556/676142066.py:16: UserWarning: Skipping entity [66, 70, DESC] in the following text because the character span 'lobé' does not align with token boundaries:

'folioles rhomboïdales , au sommet obtus , parfois grossièrement bilobé , sessiles , coriaces ; à la base le lobe antérieur est arrondi , le lobe postérieur atténué .'

  warnings.warn(msg)
C:\Users\NEW-USER\AppData\Local\Temp/ipykernel_23556/676142066.py:16: UserWarning: Skipping entity [7, 11, DESC] in the following text because the character span 'trie' does not align with token boundaries:

"dissymétrie de la foliole tant à la base qu'au sommet , très marquée par la nervure principale en diagonale ."

  warnings.warn(msg)
C:\Users\NEW-USER\AppData\Local\Temp/ipykernel_23556/676142066.py:16: UserWarning: Skipping entity [31, 37, DESC] in the following text because the character span 'ovales' does not align with token boundaries:

"gousses ligneuses , plates , obovales , i5-2i x

C:\Users\NEW-USER\AppData\Local\Temp/ipykernel_23556/676142066.py:16: UserWarning: Skipping entity [292, 297, DESC] in the following text because the character span 'cordé' does not align with token boundaries:

'digitées , glabres , plutôt minces : stipules intrapétiolaires , longues de 2 cm , adhérentes à la base du pétiole ; pétiole robuste , arrondi , strié , atteignant 20 cm de long ; 5-g folioles , articulées sur un pétiolule long de 5-7 cm ; limbe ovale-lancéolé , 6-18 x 14-32 cm , cordé , subcordé ou tronqué à la base , longuement — i5 —'

  warnings.warn(msg)
C:\Users\NEW-USER\AppData\Local\Temp/ipykernel_23556/676142066.py:16: UserWarning: Skipping entity [180, 188, DESC] in the following text because the character span 'triforme' does not align with token boundaries:

"fleurs obovoïdes , hautes de mni avant l'anthèse ; récep-tacle obconique , large de 2,5 mm pour 2 mm de haut , marge du calice épaisse , légèrement étalée , entière ; corolle calyptriforme , se détachant en en

In [428]:
%%time

VALID = []
for f in folder_test:
    path = 'data/' + f
    for i in range(len(os.listdir(path))):
        with open(path+'/'+ str(i+1) +'.txt', 'r') as fp:
            lines = fp.read()
            sentences = sent_tokenize(lines)

            for sent in sentences:
                sent = sent.replace("\n", "").lower()
                sent = preprocess(sent)
                sent = " ".join(word_tokenize(sent))
                res = descripteur(sent) + organe_(sent) + measure(sent) + list(set(color(sent)))
                if len(res) != 0:
                    res = fix_overlap(res)
                    res = {'entities' : res}
                    result = (sent , res)
                    VALID.append(result)


Wall time: 17.8 s


In [429]:
convert("fr", VALID, "ner_data/valid.spacy")

C:\Users\NEW-USER\AppData\Local\Temp/ipykernel_23556/676142066.py:16: UserWarning: Skipping entity [204, 215, DESC] in the following text because the character span 'trimoinedbd' does not align with token boundaries:

"7/ pour toute autre question relative à la réutilis ation des documents numérisés par le mnhn , l'utilis ateur est invité à s'informer auprès de la direction des bibl iothèques et de la documentation : patrimoinedbd @ mnhn.fr"

  warnings.warn(msg)
C:\Users\NEW-USER\AppData\Local\Temp/ipykernel_23556/676142066.py:16: UserWarning: Skipping entity [27, 35, DESC] in the following text because the character span 'tribuent' does not align with token boundaries:

'ces touffes de feuilles contribuent à donner aux cîmes un aspect caractéristique .'

  warnings.warn(msg)
C:\Users\NEW-USER\AppData\Local\Temp/ipykernel_23556/676142066.py:16: UserWarning: Skipping entity [100, 107, DESC] in the following text because the character span 'allèles' does not align with token boundaries:

C:\Users\NEW-USER\AppData\Local\Temp/ipykernel_23556/676142066.py:16: UserWarning: Skipping entity [43, 48, COLOR] in the following text because the character span 'verts' does not align with token boundaries:

'sépales oblongs , obtus , 5-6 mm long , couverts de poils en navette extérieurement , glabres intérieurement .'

  warnings.warn(msg)
C:\Users\NEW-USER\AppData\Local\Temp/ipykernel_23556/676142066.py:16: UserWarning: Skipping entity [60, 65, DESC] in the following text because the character span 'trice' does not align with token boundaries:

"le fruit ne contient qu'une graine , remarquable par la cicatrice basale circulaire ."

  warnings.warn(msg)
C:\Users\NEW-USER\AppData\Local\Temp/ipykernel_23556/676142066.py:16: UserWarning: Skipping entity [24, 28, COLOR] in the following text because the character span 'vert' does not align with token boundaries:

"le limbe est d'abord couvert en dessous d'un tomentum brunâtre qui est rapidement caduc , mais persiste cependant assez lon

C:\Users\NEW-USER\AppData\Local\Temp/ipykernel_23556/676142066.py:16: UserWarning: Skipping entity [79, 86, DESC] in the following text because the character span 'trional' does not align with token boundaries:

'espèce des montagnes et des galeries forestières du domaine périphérique septentrional de la région guinéo-congolaise , et qui , curieusement , se retrouve au mayombé ( gabon ) et en angola dans le domaine homologue périphérique méridional .'

  warnings.warn(msg)
C:\Users\NEW-USER\AppData\Local\Temp/ipykernel_23556/676142066.py:16: UserWarning: Skipping entity [112, 121, DESC] in the following text because the character span 'trionales' does not align with token boundaries:

"au cameroun elle est signalée dans le pays bamiléké et dans l'adamaoua , c'est-à-dire au-delà des limites septentrionales de la forêt dense conti-"

  warnings.warn(msg)
C:\Users\NEW-USER\AppData\Local\Temp/ipykernel_23556/676142066.py:16: UserWarning: Skipping entity [45, 50, DESC] in the following text

C:\Users\NEW-USER\AppData\Local\Temp/ipykernel_23556/676142066.py:16: UserWarning: Skipping entity [62, 67, DESC] in the following text because the character span 'trice' does not align with token boundaries:

'2 cm long , contenant une seule graine ellipsoïde à large cicatrice ventrale .'

  warnings.warn(msg)
C:\Users\NEW-USER\AppData\Local\Temp/ipykernel_23556/676142066.py:16: UserWarning: Skipping entity [21, 26, DESC] in the following text because the character span 'trice' does not align with token boundaries:

'graines plates à cicatrice latérale et basale .'

  warnings.warn(msg)
C:\Users\NEW-USER\AppData\Local\Temp/ipykernel_23556/676142066.py:16: UserWarning: Skipping entity [40, 45, COLOR] in the following text because the character span 'verte' does not align with token boundaries:

"l'espèce type b. leptosperma a été découverte en côte d'ivoire ( a ."

  warnings.warn(msg)
C:\Users\NEW-USER\AppData\Local\Temp/ipykernel_23556/676142066.py:16: UserWarning: Skipping entity [5

C:\Users\NEW-USER\AppData\Local\Temp/ipykernel_23556/676142066.py:16: UserWarning: Skipping entity [38, 46, DESC] in the following text because the character span 'lancéolé' does not align with token boundaries:

'feuilles sessiles ; limbe obovale à oblancéolé , obtus ou arrondi au sommet et mucronulé , cunéiforme à la base , de 2,5-4,5 x 1,3-2 cm , pubescent sur les deux faces ; 3 paires de ner-vures secondaires , ascendantes , plus nettes en dessous .'

  warnings.warn(msg)
C:\Users\NEW-USER\AppData\Local\Temp/ipykernel_23556/676142066.py:16: UserWarning: Skipping entity [28, 33, DESC] in the following text because the character span 'ovale' does not align with token boundaries:

'feuilles sessiles ; limbe obovale à oblancéolé , obtus ou arrondi au sommet et mucronulé , cunéiforme à la base , de 2,5-4,5 x 1,3-2 cm , pubescent sur les deux faces ; 3 paires de ner-vures secondaires , ascendantes , plus nettes en dessous .'

  warnings.warn(msg)
C:\Users\NEW-USER\AppData\Local\Temp/ipyk

## ===============

# Tesseract OCR

In [ ]:
import os
from PIL import Image
from pdf2image import convert_from_path
import pytesseract

pytesseract.pytesseract.tesseract_cmd =r'C:\Users\NEW-USER\AppData\Local\Tesseract-OCR\tesseract.exe'
poppler_path = r"C:\Users\NEW-USER\AppData\Local\poppler-22.04.0\Library\bin"

In [ ]:
from pdf2image import convert_from_path
from pytesseract import image_to_string
from PIL import Image

In [ ]:
def convert_pdf_img(pdf_file):
    return convert_from_path(pdf_file, poppler_path = poppler_path)
def convert_img_txt(file):
    text = image_to_string(file)
    return text

def get_text_from_any_pdf(pdf_file):
    images = convert_pdf_img(pdf_file)
    final_text = ""
    for pg,img in enumerate(images):
        print(pg)
        final_text += convert_img_txt(img)
    return final_text

In [ ]:
path_pdf = 'data/Flore du Cameroun tome 10 1970 Ombellales.pdf'
pdf1 = get_text_from_any_pdf(path_pdf)
print(pdf1)